In [ ]:
import requests
import zipfile
import tarfile

# 다운로드
url = "https://archive.ics.uci.edu/static/public/385/kasandr.zip"
response = requests.get(url)
with open("kasandr.zip", "wb") as file:
    file.write(response.content)

# kasandr.zip 파일 해제
with zipfile.ZipFile("kasandr.zip", "r") as zip_ref:
    zip_ref.extractall("./kasandr_data")

# de.tar.bz2 파일 해제
with tarfile.open("./kasandr_data/de.tar.bz2", "r:bz2") as tar:
    tar.extractall("./kasandr_data", filter='data')


In [ ]:
import pandas as pd

file_path = "/content/kasandr_data/de/train_de.csv"
df = pd.read_csv(file_path, delimiter='\t')
df.head(3)


In [ ]:
# Parquet 파일로 저장
parquet_file_path = "./kasandr_data/de/train_de.parquet"
df.to_parquet(parquet_file_path, engine="pyarrow", index=False)

print(f"Parquet 파일이 저장되었습니다: {parquet_file_path}")


In [ ]:
import os
import pandas as pd

# Parquet 파일 경로 지정
parquet_file_path = "./kasandr_data/de/train_de.parquet"

# Parquet 파일 불러오기
df_parquet = pd.read_parquet(parquet_file_path)

# Parquet 파일 크기 확인
parquet_file_size = os.path.getsize(parquet_file_path)

# 파일 크기를 MB 단위로 변환
parquet_file_size_mb = parquet_file_size / (1024 * 1024)

print(f"Parquet 파일 크기: {parquet_file_size_mb:.2f} MB")
print(df_parquet.head(3))

In [ ]:
%%time

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

import xgboost as xgb


# Parquet 파일 경로
parquet_file_path = "./kasandr_data/de/train_de.parquet"

# 데이터 불러오기
df = pd.read_parquet(parquet_file_path)

# 날짜 컬럼 'utcdate'를 타임스탬프 형식으로 변환
df['utcdate'] = pd.to_datetime(df['utcdate']) # df['utcdate']를 datetime 객체로 변환
df['utcdate'] = df['utcdate'].astype(int) / 10**9  # 초 단위로 변환

# 범주형 컬럼을 scikit-learn LabelEncoder로 변환
label_encoder = LabelEncoder()
for column in ['userid', 'offerid', 'countrycode', 'category', 'merchant']:
    df[column] = label_encoder.fit_transform(df[column])

# 특징(X)과 레이블(y) 분리
X = df.drop('rating', axis=1)  # rating 열은 레이블이므로 제외
y = df['rating']  # rating 열을 레이블로 지정

# 훈련 데이터 80%, 검증 데이터 20%로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# DMatrix 생성 (XGBoost에서 데이터를 효율적으로 처리하는 데이터 구조)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)

# XGBoost CPU 모델 초기화 및 학습
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'device': 'cpu'
}
xgb_model = xgb.train(params, dtrain, num_boost_round=100)

# 테스트 데이터 확률 예측
y_test_pred = xgb_model.predict(dtest)

# 확률이 0.5 이상이면 1, 미만이면 0으로 변환
y_test_pred_binary = (y_test_pred >= 0.5).astype(int)

# 모델 평가
xgb_cpu_accuracy = accuracy_score(y_test, y_test_pred_binary)
xgb_cpu_conf_matrix = confusion_matrix(y_test, y_test_pred_binary)

print(f'XGBoost CPU Classifier 정확도: {xgb_cpu_accuracy}')
print('XGBoost CPU Classifier 혼동행렬:')
print(xgb_cpu_conf_matrix)


In [ ]:
%%time

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

import xgboost as xgb


# Parquet 파일 경로
parquet_file_path = "./kasandr_data/de/train_de.parquet"

# 데이터 불러오기
df = pd.read_parquet(parquet_file_path)

# 날짜 컬럼 'utcdate'를 타임스탬프 형식으로 변환
df['utcdate'] = pd.to_datetime(df['utcdate']) # df['utcdate']를 datetime 객체로 변환
df['utcdate'] = df['utcdate'].astype(int) / 10**9  # 초 단위로 변환

# 범주형 컬럼을 scikit-learn LabelEncoder로 변환
label_encoder = LabelEncoder()
for column in ['userid', 'offerid', 'countrycode', 'category', 'merchant']:
    df[column] = label_encoder.fit_transform(df[column])

# 특징(X)과 레이블(y) 분리
X = df.drop('rating', axis=1)  # rating 열은 레이블이므로 제외
y = df['rating']  # rating 열을 레이블로 지정

# 훈련 데이터 80%, 검증 데이터 20%로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# DMatrix 생성 (XGBoost에서 데이터를 효율적으로 처리하는 데이터 구조)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)

# XGBoost GPU 모델 초기화 및 학습
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'device': 'cuda'
}
xgb_model = xgb.train(params, dtrain, num_boost_round=100)

# 테스트 데이터 확률 예측
y_test_pred = xgb_model.predict(dtest)

# 확률이 0.5 이상이면 1, 미만이면 0으로 변환
y_test_pred_binary = (y_test_pred > 0.5).astype(int)

# 모델 평가
xgb_gpu_accuracy = accuracy_score(y_test, y_test_pred_binary)
xgb_gpu_conf_matrix = confusion_matrix(y_test, y_test_pred_binary)

print(f'XGBoost GPU Classifier 정확도: {xgb_gpu_accuracy}')
print('XGBoost GPU Classifier 혼동행렬:')
print(xgb_gpu_conf_matrix)


In [ ]:
%%time

import xgboost as xgb

import cupy as cp
import cudf
import cuml

from cuml.preprocessing import LabelEncoder
from cuml.model_selection import train_test_split
from cuml.metrics import accuracy_score, confusion_matrix


# Parquet 파일 경로
parquet_file_path = "./kasandr_data/de/train_de.parquet"

# 데이터 불러오기 (cudf로 변경)
df = cudf.read_parquet(parquet_file_path)

# 날짜 컬럼 'utcdate'를 타임스탬프 형식으로 변환
df['utcdate'] = cudf.to_datetime(df['utcdate']) # df['utcdate']를 datetime 객체로 변환
df['utcdate'] = df['utcdate'].astype('int64') // 10**9  # 초 단위로 변환

# 범주형 컬럼을 cuML LabelEncoder로 변환
label_encoder = LabelEncoder()
for column in ['userid', 'offerid', 'countrycode', 'category', 'merchant']:
    df[column] = label_encoder.fit_transform(df[column])

# 특징(X)과 레이블(y) 분리
X = df.drop('rating', axis=1)  # rating 열은 레이블이므로 제외
y = df['rating']  # rating 열을 레이블로 지정

# cuDF에서 train_test_split 사용을 위한 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# XGBoost DMatrix로 변환
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)

# XGBoost GPU 모델 초기화 및 학습
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'device': 'cuda',
}
xgb_model = xgb.train(params, dtrain, num_boost_round=100)

# 예측 (확률 값)
y_test_pred = xgb_model.predict(dtest)

# 확률 값(0~1)을 이진 클래스(0, 1)로 변환
y_test_pred_binary = (y_test_pred > 0.5).astype(int)

# 모델 평가
xgb_rapids_accuracy = accuracy_score(y_test, y_test_pred_binary)
xgb_rapids_conf_matrix = confusion_matrix(y_test, y_test_pred_binary)

print(f'RAPIDS + XGBoost GPU Classifier 정확도: {xgb_rapids_accuracy}')
print('RAPIDS + XGBoost GPU Classifier 혼동행렬:')
print(xgb_rapids_conf_matrix)
